## 1.1 MyTime

Định ra class MyTime để lưu lại thời gian trong ngày. Class định ra sẽ trông như thế này:

In [1]:
class MyTime:
    def __init__(self, hrs=0, mins= 0, secs=0):
        """ Create a MyTime object initialized to hrs, mins, secs """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs

In [2]:
tim1 = MyTime(11, 59, 30)
tim1

In [3]:
tim1.hours

11

## 1.2 Pure functions

Viết một hàm gọi là `add_time` để tổng thời gian giữa 2 objects. Cái này để chứng minh cho 2 loại function là pure function và modifier

In [4]:
def add_time(t1, t2):
    h = t1.hours + t2.hours
    m = t1.minutes + t2.minutes
    s = t1.seconds + t2.seconds
    sum_t = MyTime(h, m, s)
    return sum_t

function này được tạo bởi object MyTime và gọi là **pure function** vì nó không chỉnh sửa gì bất kì đối tượng nào được pass làm thông số.

In [11]:
current_time = MyTime(9, 14, 30)
bread_time = MyTime(3, 35, 0)
done_time = add_time(current_time, bread_time)

In [6]:
print("{}:{}:{}".format(done_time.hours, done_time.minutes, done_time.seconds))

12:49:30


Kết quả đúng nhưng ở các case khác thì kết quả sẽ không đúng. Như trong trường hợp các case có các attributes như hours, minutes, seconds mà tổng giữa t1 và t2 lớn hơn 60. Ở dưới là một phiên bản tốt hơn.

In [7]:
def add_time(t1, t2):
    h = t1.hours + t2.hours
    m = t1.minutes + t2.minutes
    s = t1.seconds + t2.seconds
    
    if s>= 60:
        s -= 60
        m += 1
    
    if m >= 60:
        m -= 60
        h +=1
        
    sum_t = MyTime(h, m, s) 
    return sum_t

Hàm này đã trở nên lớn nhưng vẫn không hiệu quả với mọi trường hợp.

## 1.3 Modifiers

Sẽ có những lúc hữu ích cho một hàm có thể chỉnh sửa một hoặc nhiều đối tượng khi đưa nó vào làm thông số. Những hàm hoạt động theo kiểu đó gọi là **modifiers**

In [8]:
def increment(t, secs):
    t.seconds += secs
    
    if t.seconds >= 60:
        t.seconds -= 60
        t.minutes +=1
    
    if t.minutes >= 60:
        t.minutes -=60
        t.hours += 1    

Dòng đầu là một phép toán đơn giản, phần còn lại là cho các trường hợp đặc biệt mà ta gặp ở trên. Vấn đề là nếu các attribute này lớn hơn rất nhiều so với 60 thì nó lại chỉ tính có 1 lần. Vậy nên phù hợp là thay `if` bằng `while`

In [9]:
def increment(t, secs):
    t.seconds += secs
    
    while t.seconds >= 60:
        t.seconds -= 60
        t.minutes +=1
    
    while t.minutes >= 60:
        t.minutes -=60
        t.hours += 1  

Hàm này đến giờ khá đúng nếu seconds không âm. Nhưng nếu thời gian vượt quá 23 giờ thì nó vẫn chưa phải giải pháp tốt.

## 1.4 Converting increment to a method

Một người lập trình định hướng đối tượng sẽ muốn sử dụng các hàm hoạt động với các object thuộc MyTime vào class MyTime

In [10]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs
    
    def increment(self, seconds):
        self.seconds += seconds
        
        while self.seconds >= 60:
            self.seconds -=60
            self.minutes +=1
            
        while self.minutes >= 60:
            self.minutes -= 60
            self.hours += 1        

Việc chuyển đổi này khá máy móc. Ta đưa hàm này vào trong lúc định ra class MyTime. Và thay tham số đầu tiên là self. Thử với current_time như ở trên, thêm 500 giây.

In [13]:
current_time = MyTime(9, 14, 30)
current_time.increment(500)
print("{}:{}:{}".format(current_time.hours, current_time.minutes, current_time.seconds))

9:22:50


## 1.5 An “Aha!” insight

Thông thường có một cái nhìn insight hơn trong bài toán thì có thể giúp việc lập trình dễ hơn. Trong trường hợp này, insight của object MyTime là 3 chữ số base trên 60. Được chia làm 3 cột hours, minutes và seconds. Các quan sát này gợi ý cho một cách tiếp cận khác của cả vấn đề. Đó là convert MyTime object thành 1 cột seconds.

In [4]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):     
        
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        # Calculate total seconds to represent
        
        totalsecs = hrs*3600 + mins*60 + secs
        self.hours = totalsecs // 3600
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs //60
        self.seconds = leftoversecs % 60
    
    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours*3600 + self.minutes*60 + self.seconds

Khối code từ line 10 đến 14 để chuẩn hóa lại các giá trị thời gian.

Ở trong lập trình định hướng đối tượng, chúng ta thật sự muốn gói dữ liệu và hoạt động lại cùng nhau. Nên chúng ta sẽ muốn làm như vậy trong class MyTime. Một giải pháp tốt là viết lại khởi tạo class, điều đó có thể giúp đương đầu với những giá trị _phút_ và _giây_ nằm ngoài chuẩn hóa. Ví dụ 3 hours 80 minutes 121 seconds thì nên được chuẩn hóa về 4 hours 22 minutes 1 second

In [7]:
nomalized_time = MyTime(3, 80, 121)
print("{}:{}:{}".format(nomalized_time.hours, nomalized_time.minutes, nomalized_time.seconds))

4:22:1


In [5]:
def add_time(t1, t2):
    secs = t1.to_seconds() + t2.to_seconds()
    return MyTime(0, 0, secs)

In [6]:
current_time = MyTime(9, 14, 30)
bread_time = MyTime(3, 35, 0)
done_time = add_time(current_time, bread_time)
print("{}:{}:{}".format(done_time.hours, done_time.minutes, done_time.seconds))

12:49:30


## 1.6 Another example